In [1]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import cv2
from glob import glob
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization, Input
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

# Define paths to your dataset
TRAIN_DIR = os.path.join(os.getcwd(), 'train')
VAL_DIR = os.path.join('val')


In [2]:
# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)


In [12]:
# Load Data
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(200, 200),
    batch_size=32,
    class_mode=None,
    shuffle=True
)

validation_generator = test_datagen.flow_from_directory(
    VAL_DIR,
    target_size=(200, 200),
    batch_size=32,
    class_mode=None,
    shuffle=False
)

def get_file_paths(directory):
    return glob(os.path.join(directory, '*/*.jpg')) 
# Obtain file paths
train_filepaths = get_file_paths(TRAIN_DIR)
val_filepaths = get_file_paths(VAL_DIR)

# Preprocess labels function
def preprocess_labels(filepaths):
    ages = []
    nationalities = []
    for filepath in filepaths:
        filename = os.path.basename(filepath)
        parts = filename.split('_')
        if len(parts) != 4:
            continue
        age = int(parts[0])
        race = int(parts[2])
        # Filter out ages outside the range [10, 60]
        if age < 10 or age > 60:
            continue
        ages.append(age)
        nationalities.append(race)
    return np.array(ages), np.array(nationalities)

# Call preprocess_labels with correct arguments
y_train_age, y_train_nat = preprocess_labels(train_filepaths)
y_val_age, y_val_nat = preprocess_labels(val_filepaths)

Found 33488 images belonging to 2 classes.
Found 33488 images belonging to 2 classes.


In [13]:
# Model Architecture
input_shape = (200, 200, 3)
inputs = Input(shape=input_shape)

x = Conv2D(32, (3, 3), activation='relu')(inputs)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)

x = Conv2D(64, (3, 3), activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)

x = Conv2D(128, (3, 3), activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)

x = Flatten()(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

age_output = Dense(1, activation='linear', name='age_output')(x)
nat_output = Dense(5, activation='softmax', name='nat_output')(x)  # Assuming 5 nationalities in the dataset

model = Model(inputs=inputs, outputs=[age_output, nat_output])


In [17]:
# Compile Model
model.compile(optimizer='adam',
              loss={'age_output': 'mean_squared_error', 'nat_output': 'categorical_crossentropy'},
              metrics={'age_output': 'mae', 'nat_output': 'accuracy'})


# Callbacks
checkpoint = ModelCheckpoint("model_weights.weights.h5", monitor="val_nat_output_accuracy",
                             save_weights_only=True, mode="max", verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, min_lr=0.00001, mode='auto')

callbacks = [checkpoint, reduce_lr]



In [19]:
# Train the Model
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator,
    callbacks=callbacks
)


Epoch 1/20


TypeError: 'NoneType' object is not iterable